In [71]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [72]:
# datasets importen
df = pd.read_csv('../resources/dataset_weather/weather_accidents.csv')

In [73]:
# Filter the df to detroit
df_detroit = df[df['City'] == 'Detroit']

# Drop irralevant columns
df_detroit = df_detroit.drop(columns=['Severity', 'City'])

In [74]:
# Count the entries per day
accident_count = df_detroit['Date'].value_counts().sort_index()

In [75]:
# Merge count with df
df_detroit_count = pd.merge(df_detroit, accident_count, left_on=['Date'], right_on=[accident_count.index])

In [76]:
# Group dataframe by date
df_detroit_count = df_detroit_count.groupby('Date').mean()
df_detroit_count = df_detroit_count.reset_index()

In [77]:
df_detroit_count['Date'] = pd.to_datetime(df_detroit_count['Date'])
df_detroit_count['Date'] = df_detroit_count['Date'].dt.month_name()


In [78]:
# df_detroit_count['Temp_bins'] = pd.cut(df_detroit_count['Temperature'], [0,275,1000], labels=['Freezing', 'Not Freezing'])
df_detroit_count['Temp_bins'] = pd.cut(df_detroit_count['Temperature'], [0,275,1000], labels=['Freezing', 'Not Freezing'])
df_detroit_count['Humidity_bins'] = pd.qcut(df_detroit_count['Humidity'], q=3, labels=['Low','Medium','High'])
df_detroit_count['Wind_Speed_bins'] = pd.qcut(df_detroit_count['Wind_Speed_km'], q=3, labels=['Low','Medium','High'])
df_detroit_count['count_bins'] = pd.qcut(df_detroit_count['count'], q=3, labels=['Low','Medium','High'])

In [79]:
dimensions=[
    {'label': 'Month', 'values': df_detroit_count['Date']},
    {'label': 'Temperature', 'values': df_detroit_count['Temp_bins'], 'categoryarray': ['Not Freezing', 'Freezing'] ,"categoryorder": "array"},
    {'label': 'Humidity', 'values': df_detroit_count['Humidity_bins'], 'categoryarray': ['High','Medium', 'Low'] ,"categoryorder": "array"},
    {'label': 'Wind Speed', 'values': df_detroit_count['Wind_Speed_bins'], 'categoryarray': ['High','Medium', 'Low'] ,"categoryorder": "array"},
    {'label': 'Number of Accidents', 'values': df_detroit_count['count_bins'], 'categoryarray': ['High','Medium', 'Low'] ,"categoryorder": "array"}
]

color_values = dimensions[-1]['values']
unique_values = list(set(color_values))
color_map = {value: idx for idx, value in enumerate(unique_values)}
color_scale = ['rgb(102,194,165)', 'rgb(252,141,98)', 'rgb(141,160,203)']

# Map the color values to the color indices
line_colors = [color_map[value] for value in color_values]


# Create the parcats chart
fig = go.Figure(data = [go.Parcats(
    dimensions=dimensions,
    line={'color': line_colors, 'colorscale': color_scale}
)])
fig.update_layout(
    title='Number of Accidents in Different Weather',
    height=600)

# Show the diagram
fig.show()